In [ ]:
import telebot
import requests
import re
from time import sleep
from telebot import types
import sqlite3

In [ ]:
bot = telebot.TeleBot('632061718:AAEoanuGDBBOkqEyUf3pk7xPFtYxb7-O-Ds', threaded=False)

In [ ]:
def make_query(*params):
    conn = sqlite3.connect('Ex.db')
    c = conn.cursor()
    c.execute(*params)
    retur = c.fetchall()
    conn.commit()
    conn.close()
    return retur
 
def nickname_define(message):
 
    name, surname = message.from_user.first_name, message.from_user.last_name
    if name:
        nickname = f'{surname}_{name}' if surname else name
    else:
        nickname = surname if surname else 'Somebody'
 
    return nickname
 
@bot.message_handler(commands=['start'])
def start(message):
    person = nickname_define(message)
    start_text = 'start'
    #Audio Button
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    markup.row('Мои аудиозаписи')
    bot.send_message(message.chat.id, start_text, reply_markup=markup)    

@bot.message_handler(commands=['help'])
def help(message):
    bot.send_message(message.chat.id, 'help')

@bot.message_handler(content_types=['text'])
def text(message):
    t = message.text
    if t == 'Мои аудиозаписи':
        url_preparing(message)

def extra(download_site):
    download_text = requests.get(download_site).text
    SongBlockList = re.findall('artist=(.+)&title=(.+)&url=', download_text)
    LinksList = re.findall('<a href="/download.php(.+)>', download_text)
 
    DownloadLinkReq = []
    for link in LinksList:
        DownloadLinkReq.append('https://vrit.me//download.php?artist=' + link[8:-1])
    return SongBlockList, DownloadLinkReq, download_text
 
def url_preparing(message):
    #Feed Button
    markup = types.InlineKeyboardMarkup()
    btn_feed= types.InlineKeyboardButton(text='Feed', url='https://vk.com/feed')
    markup.add(btn_feed)
    ask_text = 'You can get to your profile faster, using the feed link'
    sent = bot.send_message(message.chat.id, ask_text, reply_markup = markup)
    bot.register_next_step_handler(sent, url_parsing)



In [ ]:
def url_parsing(message):
    check_val = re.findall('https://vk.com/\w+', message.text)
    check_val_unsafety = re.findall('vk.com/\w+', message.text)
    user_id = ''
    id_parsing_link = 'http://showid.ru/names/'
    correct_info = 'Your link is correct. Now i am parsing it'
    incorrect_info = 'Your link is incorrect, I can\'t parse it. Please, check it again'
    if (len(check_val) == 1 and (check_val[0] == message.text)):
        user_id = message.text[15:]
        bot.send_message(message.chat.id, correct_info)
    elif len(check_val_unsafety) == 1 and (check_val_unsafety[0] == message.text):
        user_id = message.text[7:]
        bot.send_message(message.chat.id, correct_info)
    else:
        bot.send_message(message.chat.id, incorrect_info)
    if len(user_id) != 0:
        id_parsing_link += user_id
        page = requests.get(id_parsing_link)
        req = page.text
        number_id_req = re.findall('<p><strong>ID:</strong> \d+</p>', req)
        number_id = number_id_req[0][24:-4]
        download_site = "https://vrit.me/audios" + number_id
    SongBlockList, DownloadLinkReq, download_text = extra(download_site)
    if len(SongBlockList) == 0: SongBlockList, DownloadLinkReq, download_text = extra(download_site)
 
    if 'Для скачивания музыки разрешите доступ к своим аудиозаписям!' in download_text:
        bot.send_message(message.chat.id, "Your audios are closed😞. Please, add something, and repeat")  
 
    elif len(SongBlockList) == 0:
        bot.send_message(message.chat.id, "Your audios are empty😞. Please, open them, and repeat")
 
    else:
        SongBlockList = [SongBlockList[i][0]+'-'+SongBlockList[i][1] for i in range(len(SongBlockList))]
        ListOfSongs = list()
        for i in range(len(SongBlockList)//5 + (1 if len(SongBlockList)%5 > 0 else 0)):
            if i == len(SongBlockList)//5 + (1 if len(SongBlockList)%5 > 0 else 0)-1:
                ListOfSongs.append(SongBlockList[i*5:])
            else:
                ListOfSongs.append(SongBlockList[i*5:i*5+5])
        new = list()
        for x in ListOfSongs[-1]:
            if x != '': new.append(x)
        ListOfSongs[-1] = new
        nickname = nickname_define(message)
 
        make_query(f"DROP TABLE IF EXISTS {nickname};")
        make_query(f"CREATE TABLE {nickname} (Ua_ID INTEGER PRIMARY KEY AUTOINCREMENT, Songs text, Links text);")
        make_query(f'insert into {nickname} (Songs, Links) values (?, ?)', (';;;'.join([';;'.join(x) for x in ListOfSongs]), ';;;'.join(DownloadLinkReq)))
 
        markup = types.InlineKeyboardMarkup()
        for i in range(len(ListOfSongs[0])):
            btn = types.InlineKeyboardButton(text=f'{ListOfSongs[0][i]}', url=DownloadLinkReq[i])
            markup.add(btn)
        btn1 = types.InlineKeyboardButton(text='⬅️', callback_data='⬅️'+';'+'0'+';'+nickname)
        btn2 = types.InlineKeyboardButton(text='➡️', callback_data='➡️'+';'+'0'+';'+nickname)
        markup.add(btn1, btn2)
        bot.send_message(message.chat.id, text="pesni", reply_markup=markup)
 
 
@bot.callback_query_handler(func=lambda c: True)
def inline(c):
 
    if '⬅️' in c.data or '➡️' in c.data:
        act, page, nickname = (c.data).split(';')
        page = int(page)
        songs = [x.split(';;') for x in (make_query(f'select Songs from {nickname}')[0][0]).split(';;;')]
        if act == '➡️':
            page = (page+1)%(len(songs))
        else:
            page = (page-1)%(len(songs))
 
 
        links = (make_query(f'select Links from {nickname}')[0][0]).split(';;;')
        markup = types.InlineKeyboardMarkup()
        for i in range(len(songs[page])):
            btn = types.InlineKeyboardButton(text=f'{songs[page][i]}', url=links[5*page+i])
            markup.add(btn)
        btn1 = types.InlineKeyboardButton(text='⬅️', callback_data='⬅️'+';'+str(page)+';'+nickname)
        btn2 = types.InlineKeyboardButton(text='➡️', callback_data='➡️'+';'+str(page)+';'+nickname)
        markup.add(btn1, btn2)
 
        bot.edit_message_text(chat_id=c.message.chat.id, message_id=c.message.message_id, text='pesni', reply_markup=markup)
 
 


In [ ]:
def main():
    bot.polling(none_stop=True, interval=0)
 
 
if __name__ == '__main__':
    while True:
        try:
            main()
        except Exception as e:
            print(e)
            sleep(1)